# KPIs

KPI.01

Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Tasa de homicidios en siniestros viales = (Número de homicidios en siniestros viales / Población total) * 100,000


In [ ]:
def determinar_semestre(mes):
    if MM <= 6:
        return 1
    else:
        return 2


df_siniestros["SEMESTRE"] = df_siniestros["MM"].apply(determinar_semestre)

KPI.02

Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Tasa de homicidios en siniestros viales = (Número de homicidios en siniestros viales / Población total) * 100,000


KPI.03

Reducir en un 10% la tasa de homicidios en siniestros viales de los últimos seis meses, en CABA, en comparación con la tasa de homicidios en siniestros viales del semestre anterior.

Tasa de homicidios en siniestros viales = (Número de homicidios en siniestros viales / Población total) * 100,000
